<a href="https://colab.research.google.com/github/proteinphysiologylab/frustratometeR/blob/master/FrustratometeR_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Local Energetic Frustration**
In this colab we will learn to calculate and analyze frustration patterns protein structures. For this we will use new functionalities of the frustratometer (R package). We will also analyze frustration in molecular dynamics simulations.

```
# Esto tiene formato de código
```



### **We are going to test the Frustratometer.**
We are going to install the minimum libraries to be able to run the **frustratometer** in colab!



In [ ]:
#Install biopython and visualization
!pip install biopython py3Dmol
#Downgrade rpy para correr frustraR
!pip install rpy2==3.5.1

Installing dependencies (this part takes around 40 mins). Check that in the line that contains "KEY_MODELLER=XXX", "XXX" needs to be replaced by a valid Modeller licence.

In [ ]:
!apt update -y
!apt install wget mc r-base python3 python3-pip pymol libmagick++-dev libcurl4-openssl-dev libssl-dev libgit2-dev -y
!apt install libcurl4-gnutls-dev libxml2-dev -y
!python3 -m pip install numpy biopython leidenalg
!apt install software-properties-common -y
!wget https://salilab.org/modeller/9.25/modeller_9.25-1_amd64.deb
!env KEY_MODELLER=XXX dpkg -i modeller_9.25-1_amd64.deb
!Rscript -e "install.packages('ggrepel')"
!Rscript -e "install.packages('igraph')"
!Rscript -e "install.packages('dplyr')"
!Rscript -e "install.packages('FactoMineR')"
!Rscript -e "install.packages('Hmisc')"`
!Rscript -e "install.packages('argparse')"
!Rscript -e "install.packages('leiden')"
!Rscript -e "install.packages('magick')"
!Rscript -e "options(timeout=9999999)"
!Rscript -e "devtools::install_github('proteinphysiologylab/frustratometeR')"

Now, we are going to **donwload two proteins** which are two hemoglobins to which we are going to calculate the frustration at the contact level (mutational and configurational). The files will be stored in /content/

In [ ]:
#Loading the R extension interactively
%load_ext rpy2.ipython

In [ ]:
!wget http://www.rcsb.org/pdb/files/1fhj.pdb -O 1fhj.pdb
!wget http://www.rcsb.org/pdb/files/2dn1.pdb -O 2dn1.pdb
!wget http://www.rcsb.org/pdb/files/1m6k.pdb -O 1m6k.pdb

####Visualization of the structures

In [ ]:
#Visualizing the heterotetramer
import py3Dmol
#First we assign the py3Dmol.view as view
view=py3Dmol.view()
#The following lines are used to add the addModel class
#to read the PDB files of chain B and C
view.addModel(open('/content/1fhj.pdb', 'r').read(),'pdb')
#Zooming into all visualized structures 
view.zoomTo()
#Here we set the background color as white
view.setBackgroundColor('white')
#Here we set the visualization style for chain B and C
view.setStyle({'chain':'A'},{'cartoon': {'color':'purple'}})
view.setStyle({'chain':'B'},{'cartoon': {'color':'green'}})
view.setStyle({'chain':'C'},{'cartoon': {'color':'yellow'}})
view.setStyle({'chain':'D'},{'cartoon': {'color':'blue'}})
#And we finally visualize the structures using the command below
view.show()

####Frustration calculation in heterocomplexes

We will generate the **R script** to frustrate all the pdbs out there **in a directory**!




In [ ]:
#create a folder to save the frustration results (/content/Results)
!mkdir Results

In [ ]:
%%R
library(frustratometeR)

dir_frustration(PdbsDir = "/content/", Mode = "configurational", ResultsDir ="/content/Results/")

In [ ]:
from IPython.display import Image
Image('/content/Results/1fhj.done/Images/1fhj_configurational_map.png', width=600, height=400)

In [ ]:
from IPython.display import Image
Image('/content/Results/2dn1.done/Images/2dn1_configurational_map.png', width=600, height=400)

There are many **in vitro experiments on enzymatic proteins** that show that, if a residue belonging to the active site is mutated, **stability increases but this directly impacts on a reduction of enzymatic function**.
Can we capture this through a frustration analysis?

Let's take a look at an example, from the paper Characterization of the Global Stabilizing Substitution A77V and Its Role in the Evolution of CTX-M β-Lactamases

'Amino acid substitutions in the active site of CTX-M enzymes may change their activity and substrate specificity at the cost of stability. The A77V substitution is a stabilizing mutation that compensates for a loss in stability associated with substitutions that alter catalytic activity. The A77V substitution increases the overall thermal stability of the enzyme, allowing it to absorb a greater number of deleterious destabilizing mutations and increasing its access to alternate evolutionary paths'

https://journals.asm.org/doi/full/10.1128/AAC.00618-15

Let's get started **frustrating a beta-lactamase** (PdbID 1m6k Chain A).

In [ ]:
%%R
library(frustratometeR)


# Calculate frustration for a structure to be downloaded from the PDB
Pdb_sing <- calculate_frustration(PdbID = '1m6k', Mode = "mutational", ResultsDir =  "/content/Results/", Chain="A", Graphics = TRUE)


In [ ]:
from IPython.display import Image
Image('/content/Results/1m6k_A.done/Images/1m6k_A_mutational_map.png', width=600, height=400)

With the new **functionalities of the FrustraR** we can predict the frustration of a **point mutation** for all frustration modes (mutational, configurational and singleresidue) in a very easy way.


In [ ]:
%%R
help(calculate_frustration)

The following lines will mutate a given residue by all the alternative aminoacid identities and evaluate how this impact in the local energetic frustration values.

In [ ]:
%%R
library(frustratometeR)

Pdb_conf <- calculate_frustration(PdbID = "1m6k", Chain="A", Mode = 'mutational', ResultsDir = "/content/Results/")
#Pdb_conf <- calculate_frustration(PdbFile = "/content/P01116-2.pdb", Chain="A", Mode = 'mutational', ResultsDir = "/content/Results/")
Pdb_conf <- mutate_res(Pdb = Pdb_conf, Resno = as.numeric(117), Chain = "A")
plot_mutate_res(Pdb = Pdb_conf, Resno = as.numeric(117), Chain = "A", Save = TRUE )

In [ ]:
from IPython.display import Image
Image('/content/Results/P01116-2_A.done/MutationsData/Images/mutational_117_threading_A.png')



## How does the frustration of a residue change during the folding process?

Let's **create a folder named Dynamic** and download a zipped file containing frames from a **molecular dynamics simulation of 1n0r protein folding**.

Let's download and unzip the frames (they are in .pdb format)

In [ ]:
!gdown 1-jr_XQYzGWA-CGrdQjE6QjRjl1MO80nN -O /content/Dynamics.zip
!unzip /content/Dynamics.zip -d /content/

In [ ]:
!touch /content/traj.pdb
!rm /content/traj.pdb
!for f in $(ls -1v /content/Dynamics/*pdb); do (cat "${f}"; echo) >> /content/traj.pdb; done

#@title ###Visualize trajectory
#define number of frames
import py3Dmol

view = py3Dmol.view()

view.addModelsAsFrames(open('/content/traj.pdb', 'r').read(),'pdb')

view.setStyle({'cartoon': {'color':'blue'}})

BB = ['C','O','N']
view.addStyle({'and':[{'resn':["GLY","PRO"],'invert':True},{'atom':BB,'invert':True}]},
                        {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
view.addStyle({'and':[{'resn':"GLY"},{'atom':'CA'}]},
                        {'sphere':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
view.addStyle({'and':[{'resn':"PRO"},{'atom':['C','O'],'invert':True}]},
                        {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})

view.zoomTo()
view.animate({'loop': "forward"})
view.show()

Al igual que antes, vamos a crear un script de .r 



In [ ]:
%%R
library('frustratometeR')
OrderList <- c()
for(i in as.numeric(1):as.numeric(40)){OrderList <- c(OrderList, paste( i, ".pdb", sep = ""))}
Dynamic_conf <- dynamic_frustration(PdbsDir = '/content/Dynamics/', OrderList = OrderList, ResultsDir = "/content/Dynamics/Results/")

# Analyzing residue Resno
Dynamic_conf <- dynamic_res(Dynamic = Dynamic_conf, Resno = as.numeric(68), Chain = "A", Graphics = TRUE)

#Visualisations
plot_dynamic_res_5Adens_proportion(Dynamic = Dynamic_conf, Resno = as.numeric(68), Chain = "A")

In [ ]:
from IPython.display import Image
Image('/content/Dynamics/Results/Dynamic_plots_res_68_A/dynamic5adens_configurational_Res68.png')


*Now* that frustration values were calculated for a given residue for all the frames we can track its evolution as a function of the frame order (usually simulation time) Let's see!
